# EFM Large-Scale Structure (LSS) Definitive Simulation (CoCalc H100 Optimized)

This notebook performs the definitive high-resolution simulation of Large-Scale Structure (LSS) formation within the Eholoko Fluxon Model (EFM) framework, optimized for a **CoCalc H100 GPU instance** with local storage. Following extensive parameter sweeps (v1-v4) that identified the natural emergent characteristic wavelength of the NLKG system, this simulation utilizes the empirically-derived optimized dimensionless parameters to robustly reproduce EFM's predicted LSS clustering scales (147 Mpc and 628 Mpc) without the need for dark matter.

This version is tailored for high-performance computing, incorporating PyTorch's mixed precision (AMP) and **TorchScript (JIT compilation) for core derivative calculations** to ensure high throughput. Robust checkpointing ensures progress is saved locally, critical for long-running frontier simulations. The simulation operates entirely in **dimensionless units**, with physical interpretations derived during post-processing.

## EFM Theoretical Grounding for LSS (S/T State, n'=1 HDS):

1.  **Single Scalar Field (φ):** All phenomena, including cosmic structure, emerge from this fundamental field [1, 2].
2.  **NLKG Equation with EFM Self-Gravity:** The equation and its parameters (optimized from sweeps) are designed to inherently drive the formation of LSS.
3.  **Harmonic Density States (HDS):** EFM predicts a base LSS scale of 628 Mpc. This simulation aims to show that the system's natural emergent dimensionless wavelength (`λ_base_sim ≈ 2.55`) directly corresponds to this 628 Mpc scale.
4.  **Seeding Aligned with Natural Emergence**: Initial conditions now explicitly seed modes that perfectly align with the system's empirically determined natural emergent wavelength, maximizing efficiency and clarity of structure formation.

## Objectives of this Definitive Run:

-   Simulate 3D LSS formation on a **750³ grid for 200,000 timesteps**.
-   Leverage **AMP and TorchScript** for core derivative calculations, and robust local checkpointing.
-   Provide definitive computational evidence for EFM's 'Fluxonic Clustering' mechanism.
-   **Rigorously quantify emergent dimensionless clustering scales** (P(k) peaks, ξ(r) features) and demonstrate their precise alignment with EFM's predicted scales.
-   **Precisely map these emergent dimensionless scales to physical clustering scales** (`628 Mpc` and `157 Mpc`) using EFM's universal scaling laws, demonstrating direct correspondence without dark matter.
-   Provide detailed analysis of non-Gaussianity (`fNL`) and internal field oscillations.

## CoCalc Environment Setup

This notebook is configured for local execution on a powerful CoCalc H100 instance. Google Drive mounting calls are removed. Data will be saved directly to the local file system.
**`torch.compile` is explicitly disabled, but `torch.jit.script` is utilized for performance.**

In [ ]:
import os
import torch
import torch.nn as nn
import gc
import psutil
from tqdm.notebook import tqdm # Use tqdm.notebook for Jupyter environments
import numpy as np
import time
from datetime import datetime
from scipy.fft import fftn, fftfreq, ifftn # Using scipy for CPU-based FFT for final analysis
import scipy.signal # For peak finding
import torch.nn.functional as F
import torch.amp as amp # Use torch.amp for autocast
import matplotlib.pyplot as plt # For plotting
import glob

# --- FIX: Explicitly disable torch.dynamo/torch.compile to prevent Triton errors ---
import torch._dynamo
torch._dynamo.config.disable = True
torch._dynamo.config.suppress_errors = True # Suppress any potential errors even if compilation is attempted
print("WARNING: torch.compile/torch.dynamo has been explicitly disabled for stability.")

# Environment setup for PyTorch CUDA memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:256' # To help with memory fragmentation
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print(f"PyTorch version: {torch.__version__}")
num_gpus_available = torch.cuda.device_count()
available_devices_list = [torch.device(f'cuda:{i}') for i in range(num_gpus_available)]
print(f"Number of GPUs available: {num_gpus_available}, Available Devices: {available_devices_list}")
if num_gpus_available > 0:
    current_gpu_device = torch.device('cuda:0')
    print(f"Using GPU 0: {torch.cuda.get_device_name(current_gpu_device)}, VRAM: {torch.cuda.get_device_properties(current_gpu_device).total_memory / 1e9:.2f} GB")
else:
    current_gpu_device = torch.device('cpu')
    print("No GPU available, running on CPU. Performance may be limited.")
print(f"System RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

# Define paths for checkpoints and data/plots - LOCAL STORAGE FOR COCALC
checkpoint_path_lss_definitive = './EFM_Simulations/checkpoints/LSS_DEFINITIVE_N750_Run/'
data_path_lss_definitive = './EFM_Simulations/data/LSS_DEFINITIVE_N750_Run/'
os.makedirs(checkpoint_path_lss_definitive, exist_ok=True)
os.makedirs(data_path_lss_definitive, exist_ok=True)
print(f"LSS Definitive Checkpoints will be saved to: {checkpoint_path_lss_definitive}")
print(f"LSS Definitive Data/Plots will be saved to: {data_path_lss_definitive}")


## Configuration for Definitive LSS Simulation (Dimensionless Units, H100 Optimized)

Parameters are set based on the results of previous parameter sweeps, identifying the most effective values for generating EFM's predicted LSS. The `N` (grid size) is set to `750` for high-resolution output.

**Key Parameters (Optimized and Aligned with Natural Emergence):**

*   `N`: Grid size. **Set to `750`** for high-resolution simulation.
*   `T_steps`: Total simulation steps. **Set to `200000`** for sufficient evolution.

*   `m_sim_unit_inv` (m in m²φ): Mass term coefficient. **Set to `0.1`** (from v2 sweep, `m=0.1` alpha=0.7 gave 2.55, aligned with original paper's context for LSS).
*   `alpha_sim` (α in αφ(∂φ/∂t)⋅∇φ): State parameter. **Set to `0.7`** (aligned with original paper's S/T state, showed consistency with 2.55 emergent wavelength).
*   `g_sim` (g in gφ³): Cubic nonlinearity coefficient. **Set to `0.1`** (consistently used, did not shift dominant wavelength in v1).
*   `k_efm_gravity_coupling` (k in 8πGkφ²): Self-gravity coupling. **Set to `0.005`** (consistently used, did not shift dominant wavelength in v1).
*   `eta_sim` (η in ηφ⁵): Quintic nonlinearity. **Set to `0.01`** (consistently used, did not shift dominant wavelength in v3).
*   `delta_sim` (δ in δ(∂φ/∂t)²φ): Dissipation term. **Set to `0.0002`** (consistently used, did not shift dominant wavelength in v3).

*   `L_sim_unit`: Dimensionless box size. Fixed at `10.0`.
*   `c_sim_unit`: Dimensionless speed of light. Fixed at `1.0`.
*   `G_sim_unit`: Dimensionless gravitational constant. Fixed at `1.0`.
*   `seeded_perturbation_amplitude`: Amplitude of seeded modes. `1.0e-3`.
*   `background_noise_amplitude`: Amplitude of general random noise. `1.0e-6`.

**Crucial: Aligned Seeded Wavenumbers**: Based on extensive parameter sweeps, the natural emergent dimensionless base wavelength (`λ_base_sim`) of the system is consistently `~2.55`. We now align the seeding with this natural behavior.
*   `k_seed_primary`: Aligned to `λ_base_sim ≈ 2.55`. Calculated as `2 * np.pi / 2.55`.
*   `k_seed_secondary`: Aligned to `λ_base_sim / 4 ≈ 0.6375` (for 157 Mpc BAO-like scale). Calculated as `2 * np.pi / 0.6375`.

This configuration aims to produce definitive, high-resolution results for EFM's LSS formation, leveraging the system's intrinsic dynamics rather than forcing external scales.

In [ ]:
config_lss_definitive = {}
config_lss_definitive['N'] = 750  # Grid size (N x N x N) - Definitive High-Resolution
config_lss_definitive['L_sim_unit'] = 10.0  # Dimensionless box size
config_lss_definitive['dx_sim_unit'] = config_lss_definitive['L_sim_unit'] / config_lss_definitive['N'] # Dimensionless spatial step

config_lss_definitive['c_sim_unit'] = 1.0  # Dimensionless speed of light
config_lss_definitive['dt_cfl_factor'] = 0.001 # Robust CFL factor
config_lss_definitive['dt_sim_unit'] = config_lss_definitive['dt_cfl_factor'] * config_lss_definitive['dx_sim_unit'] / config_lss_definitive['c_sim_unit']

config_lss_definitive['T_steps'] = 200000 # Total number of time steps

# EFM Parameters (Optimized from sweeps) 
config_lss_definitive['m_sim_unit_inv'] = 0.1 # Optimized m from sweeps, (alpha=0.7) for 2.55 emergent
config_lss_definitive['g_sim'] = 0.1          # Consistent, did not shift dominant wavelength
config_lss_definitive['eta_sim'] = 0.01         # Consistent, did not shift dominant wavelength
config_lss_definitive['k_efm_gravity_coupling'] = 0.005 # Consistent, did not shift dominant wavelength
config_lss_definitive['G_sim_unit'] = 1.0 # Consistent
config_lss_definitive['alpha_sim'] = 0.7  # Optimized alpha from sweeps, (m=0.1) for 2.55 emergent
config_lss_definitive['delta_sim'] = 0.0002 # Consistent, did not shift dominant wavelength

# Initial Conditions - NOW ALIGNED WITH NATURAL EMERGENT WAVELENGTH (lambda_base_sim ~ 2.55)
config_lss_definitive['seeded_perturbation_amplitude'] = 1.0e-3 # Amplitude of seeded sinusoidal modes
config_lss_definitive['background_noise_amplitude'] = 1.0e-6 # Amplitude of general random background noise

# Derived natural dimensionless base wavelength from sweeps
lambda_base_sim_emergent = 2.55 # Empirically determined robust emergent wavelength

# Align k-seeds with this natural emergent wavelength and its 4th harmonic
config_lss_definitive['k_seed_primary'] = 2 * np.pi / lambda_base_sim_emergent # Corresponds to lambda_base_sim
config_lss_definitive['k_seed_secondary'] = 2 * np.pi / (lambda_base_sim_emergent / 4.0) # Corresponds to lambda_base_sim / 4

config_lss_definitive['run_id'] = (
    f"LSS_DEFINITIVE_N{config_lss_definitive['N']}_T{config_lss_definitive['T_steps']}_" +
    f"m{config_lss_definitive['m_sim_unit_inv']:.1e}_alpha{config_lss_definitive['alpha_sim']:.1e}_" +
    f"g{config_lss_definitive['g_sim']:.1e}_k{config_lss_definitive['k_efm_gravity_coupling']:.1e}_" +
    f"eta{config_lss_definitive['eta_sim']:.1e}_delta{config_lss_definitive['delta_sim']:.1e}_" +
    f"ALIGNED_SEEDS_Definitive_Run"
)

config_lss_definitive['history_every_n_steps'] = 1000 # Frequency of calculating/storing diagnostics
config_lss_definitive['checkpoint_every_n_steps'] = 5000 # Frequency of saving intermediate checkpoints

print(f"--- EFM LSS Definitive Simulation Configuration ({config_lss_definitive['run_id']}) ---")
for key, value in config_lss_definitive.items():
    if isinstance(value, (float, np.float32, np.float64)):
        print(f"{key}: {value:.4g}")
    else:
        print(f"{key}: {value}")

print("\n--- Physical Scaling Interpretation ---")
print(f"The simulation's inherent dimensionless base wavelength (lambda_base_sim) is identified as ~{lambda_base_sim_emergent} units.")
print(f"This lambda_base_sim will be scaled to EFM's primary LSS scale of 628 Mpc. Thus, 1 dimensionless unit = (628 / {lambda_base_sim_emergent:.2f}) Mpc.")
print(f"Seeded primary k: {config_lss_definitive['k_seed_primary']:.4g} (lambda: {2*np.pi/config_lss_definitive['k_seed_primary']:.4g}) units")
print(f"Seeded secondary k: {config_lss_definitive['k_seed_secondary']:.4g} (lambda: {2*np.pi/config_lss_definitive['k_seed_secondary']:.4g}) units")


## Core Simulation Functions

These functions define the EFM NLKG module, the RK4 time integration, and the energy/density norm calculation. They include **checkpointing and resume logic** for robust execution within resource constraints.
**`torch.jit.script` has been applied for performance optimization, and `torch.compile` remains disabled.**

In [41]:
import os
import torch
import torch.nn as nn
import gc
import psutil
from tqdm.notebook import tqdm
import numpy as np
import time
from datetime import datetime
from scipy.fft import fftn, fftfreq, ifftn # Using scipy for CPU-based FFT for final analysis
import scipy.signal # For peak finding
import torch.nn.functional as F
import torch.amp as amp # Use torch.amp for autocast
import matplotlib.pyplot as plt # For plotting
import glob

# Enable CuDNN benchmarking (if applicable and beneficial, often done globally)
torch.backends.cudnn.benchmark = True

# --- FIX: Moved global setup here for clarity and correct device detection ---
# Ensure current_gpu_device is defined early
if torch.cuda.is_available():
    current_gpu_device = torch.device('cuda:0')
    # Set memory fraction if needed, but often not necessary and can cause issues if too aggressive
    # torch.cuda.set_per_process_memory_fraction(0.9, device=current_gpu_device) 
else:
    current_gpu_device = torch.device('cpu')

# Disable torch.compile/torch.dynamo for stability
import torch._dynamo
torch._dynamo.config.disable = True
torch._dynamo.config.suppress_errors = True
print("WARNING: torch.compile/torch.dynamo has been explicitly disabled for stability.")

# Environment setup for PyTorch CUDA memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:256' # To help with memory fragmentation
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print(f"PyTorch version: {torch.__version__}")
num_gpus_available = torch.cuda.device_count()
available_devices_list = [torch.device(f'cuda:{i}') for i in range(num_gpus_available)]
print(f"Number of GPUs available: {num_gpus_available}, Available Devices: {available_devices_list}")
print(f"Using compute device: {current_gpu_device}") # Explicitly state the selected device
print(f"System RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

# Define paths for checkpoints and data/plots
checkpoint_path_lss_definitive = './EFM_Simulations/checkpoints/LSS_DEFINITIVE_N650_Run/'
data_path_lss_definitive = './EFM_Simulations/data/LSS_DEFINITIVE_N650_Run/'
os.makedirs(checkpoint_path_lss_definitive, exist_ok=True)
os.makedirs(data_path_lss_definitive, exist_ok=True)
print(f"LSS Definitive Checkpoints will be saved to: {checkpoint_path_lss_definitive}")
print(f"LSS Definitive Data/Plots will be saved to: {data_path_lss_definitive}")

# --- FUNCTION DEFINITIONS START HERE ---

class EFMLSSModule(nn.Module):
    """
    EFM Module for the NLKG equation for LSS, using dimensionless parameters.
    Methods like nlkg_derivative_lss are compiled by torch.jit.script(instance) after init.
    """
    def __init__(self, dx, m_sq, g, eta, k_gravity, G_gravity, c_sq, alpha_param, delta_param):
        super(EFMLSSModule, self).__init__()
        self.dx = dx
        self.m_sq = m_sq
        self.g = g
        self.eta = eta
        self.k_gravity = k_gravity
        self.G_gravity = G_gravity
        self.c_sq = c_sq
        self.alpha_param = alpha_param
        self.delta_param = delta_param
        # Stencil for Laplacian
        stencil_np = np.array([[[0,0,0],[0,1,0],[0,0,0]],
                               [[0,1,0],[1,-6,1],[0,1,0]],
                               [[0,0,0],[0,1,0],[0,0,0]]], dtype=np.float32)
        # Store stencil as a buffer, so it's part of the module's state and graph
        self.register_buffer('stencil', torch.from_numpy(stencil_np / (dx**2)).to(torch.float16).view(1, 1, 3, 3, 3))

    def conv_laplacian(self, phi_field: torch.Tensor, halo_left=None, halo_right=None):
        phi_reshaped = phi_field.unsqueeze(0).unsqueeze(0)  # Shape: [1, 1, X, Y, Z]
        if halo_left is not None and halo_right is not None:
            phi_padded = torch.cat([halo_left.unsqueeze(0).unsqueeze(0), phi_reshaped, halo_right.unsqueeze(0).unsqueeze(0)], dim=2)
            # F.conv3d itself is highly optimized. No need for torch.compile on it here.
            laplacian = F.conv3d(phi_padded, self.stencil, padding=(1,1,1))
            # Correct slicing after padding for subdomain. Assuming padding of 1 on each side.
            # The input to conv3d is phi_padded which includes halos. The output is laplacian of that. 
            # We need to return only the laplacian corresponding to the *original* phi_field, 
            # but conv_laplacian computes it for the padded region. 
            # With padding=(1,1,1), the output size matches input size if input includes halos. 
            # The issue here is how conv3d handles the input and output sizes when padded. 
            # For a 3D conv with padding 1, output dim is input_dim. 
            # If phi_padded includes the halo, then the convolution is applied over the whole extended tensor. 
            # The laplacian for the 'real' part of the subdomain is directly extracted after the convolution. 
            # No slicing is needed if padding ensures output matches desired central region length. 
            # Let's verify original conv3d padding=0 case: output = input - kernel + 1. If kernel is 3, output = input - 2. 
            # If padding is 1, output = input + 2*padding - kernel + 1 = input + 2 - 3 + 1 = input. So output size matches. 
            # Thus, we return the *whole* laplacian for the subdomain, which now effectively includes boundary computations.
            return laplacian.squeeze(0).squeeze(0) # Corrected: no slicing needed if padding handled correctly by conv3d
        else:
            phi_padded = F.pad(phi_reshaped, (1,1,1,1,1,1), mode='circular')
            laplacian = F.conv3d(phi_padded, self.stencil, padding=0)
            return laplacian.squeeze(0).squeeze(0)

    @torch.jit.script # Re-applying torch.jit.script here, hoping the conv_laplacian call now works
    def nlkg_derivative_lss(self, phi: torch.Tensor, phi_dot: torch.Tensor, halo_left=None, halo_right=None):
        # Removed torch.no_grad() from here as it's handled by the outer loop or update_phi_rk4_lss
        lap_phi = self.conv_laplacian(phi, halo_left, halo_right) # This call to another method is problematic for standalone @jit.script
        
        # Use FP32 for nonlinear terms to prevent overflow, then cast back to FP16 at the end for phi_ddot
        phi_f32 = phi.to(torch.float32)
        phi_dot_f32 = phi_dot.to(torch.float32)
        
        potential_force = self.m_sq * phi_f32 + self.g * torch.pow(phi_f32, 3) + self.eta * torch.pow(phi_f32, 5)
        grad_phi_x = (torch.roll(phi_f32, shifts=-1, dims=0) - torch.roll(phi_f32, shifts=1, dims=0)) / (2 * self.dx)
        grad_phi_y = (torch.roll(phi_f32, shifts=-1, dims=1) - torch.roll(phi_f32, shifts=1, dims=1)) / (2 * self.dx)
        grad_phi_z = (torch.roll(phi_f32, shifts=-1, dims=2) - torch.roll(phi_f32, shifts=1, dims=2)) / (2 * self.dx)
        grad_phi_abs_sq = grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2
        alpha_term = self.alpha_param * phi_f32 * phi_dot_f32 * grad_phi_abs_sq
        delta_term = self.delta_param * torch.pow(phi_dot_f32, 2) * phi_f32
        source_gravity = 8.0 * float(np.pi) * self.G_gravity * self.k_gravity * torch.pow(phi_f32, 2)
        
        # Clip terms to prevent overflow BEFORE summation
        potential_force = torch.clamp(potential_force, min=-1e10, max=1e10)
        alpha_term = torch.clamp(alpha_term, min=-1e10, max=1e10)
        delta_term = torch.clamp(delta_term, min=-1e10, max=1e10)
        source_gravity = torch.clamp(source_gravity, min=-1e10, max=1e10)
        
        # Compute phi_ddot in float32, then cast to float16
        phi_ddot = (self.c_sq * lap_phi.to(torch.float32) - potential_force + \
                    alpha_term + delta_term + source_gravity).to(torch.float16)
        # Clip phi_ddot to prevent extreme values
        phi_ddot = torch.clamp(phi_ddot, min=-1e5, max=1e5)
        
        return phi_dot, phi_ddot

# update_phi_rk4_lss itself is called by the Python loop.
# The internal call to nlkg_derivative_lss will use the TorchScripted method of efm_model.
def update_phi_rk4_lss(phi_current: torch.Tensor, phi_dot_current: torch.Tensor,\
                       dt: float, model_instance: nn.Module,\
                       halo_left=None, halo_right=None) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Updates phi and phi_dot using the RK4 method for one time step.
    `model_instance.nlkg_derivative_lss` is expected to be TorchScript compiled internally by scripting the module.
    """
    with amp.autocast(device_type=phi_current.device.type, dtype=torch.float16):
        k1_v, k1_a = model_instance.nlkg_derivative_lss(phi_current, phi_dot_current, halo_left, halo_right)
        # Check for NaN/Inf after each k-stage to catch instability early
        if torch.any(torch.isnan(k1_v) | torch.isinf(k1_v) | torch.isnan(k1_a) | torch.isinf(k1_a)):
            raise ValueError("NaN/Inf detected in k1 stage")

        phi_temp_k2 = phi_current + 0.5 * dt * k1_v
        phi_dot_temp_k2 = phi_dot_current + 0.5 * dt * k1_a
        k2_v, k2_a = model_instance.nlkg_derivative_lss(phi_temp_k2, phi_dot_temp_k2, halo_left, halo_right)
        if torch.any(torch.isnan(k2_v) | torch.isinf(k2_v) | torch.isnan(k2_a) | torch.isinf(k2_a)):
            raise ValueError("NaN/Inf detected in k2 stage")

        phi_temp_k3 = phi_current + 0.5 * dt * k2_v
        phi_dot_temp_k3 = phi_dot_current + 0.5 * dt * k2_a
        k3_v, k3_a = model_instance.nlkg_derivative_lss(phi_temp_k3, phi_dot_temp_k3, halo_left, halo_right)
        if torch.any(torch.isnan(k3_v) | torch.isinf(k3_v) | torch.isnan(k3_a) | torch.isinf(k3_a)):
            raise ValueError("NaN/Inf detected in k3 stage")

        phi_temp_k4 = phi_current + dt * k3_v
        phi_dot_temp_k4 = phi_dot_current + dt * k3_a
        k4_v, k4_a = model_instance.nlkg_derivative_lss(phi_temp_k4, phi_dot_temp_k4, halo_left, halo_right)
        if torch.any(torch.isnan(k4_v) | torch.isinf(k4_v) | torch.isnan(k4_a) | torch.isinf(k4_a)):
            raise ValueError("NaN/Inf detected in k4 stage")

        phi_new = phi_current + (dt / 6.0) * (k1_v + 2*k2_v + 2*k3_v + k4_v)
        phi_dot_new = phi_dot_current + (dt / 6.0) * (k1_a + 2*k2_a + 2*k3_a + k4_a)
        
        if torch.any(torch.isnan(phi_new) | torch.isinf(phi_new) | torch.isnan(phi_dot_new) | torch.isinf(phi_dot_new)):
            raise ValueError("NaN/Inf detected in final update")
            
    return phi_new, phi_dot_new

# --- FIX: compute_total_energy_lss function definition (moved up) ---
def compute_total_energy_lss(phi: torch.Tensor, phi_dot: torch.Tensor,\
                             m_sq_param: float, g_param: float, eta_param: float,\
                             dx: float, c_sq_param: float) -> float:
    """Computes the total field energy based on the EFM Lagrangian for LSS (dimensionless units)."""
    vol_element = dx**3

    phi_f32 = phi.to(dtype=torch.float32)
    phi_dot_f32 = phi_dot.to(dtype=torch.float32)

    # Use amp.autocast for this section too, as tensors are on GPU
    with amp.autocast(device_type=phi.device.type, dtype=torch.float16):
        kinetic_density = 0.5 * torch.pow(phi_dot_f32, 2)
        potential_density = (0.5 * m_sq_param * torch.pow(phi_f32, 2) +\
                             0.25 * g_param * torch.pow(phi_f32, 4) +\
                             (1.0/6.0) * eta_param * torch.pow(phi_f32, 6))

        grad_phi_x = (torch.roll(phi_f32, shifts=-1, dims=0) - torch.roll(phi_f32, shifts=1, dims=0)) / (2 * dx)
        grad_phi_y = (torch.roll(phi_f32, shifts=-1, dims=1) - torch.roll(phi_f32, shifts=1, dims=1)) / (2 * dx)
        grad_phi_z = (torch.roll(phi_f32, shifts=-1, dims=2) - torch.roll(phi_f32, shifts=1, dims=2)) / (2 * dx)

        grad_phi_abs_sq = grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2
        gradient_energy_density = 0.5 * c_sq_param * grad_phi_abs_sq

        total_energy_current_chunk = torch.sum(kinetic_density + potential_density + gradient_energy_density) * vol_element

    if torch.isnan(total_energy_current_chunk) or torch.isinf(total_energy_current_chunk):
        return float('nan')

    total_energy_val = total_energy_current_chunk.item()

    del phi_f32, phi_dot_f32, kinetic_density, potential_density, gradient_energy_density
    del grad_phi_x, grad_phi_y, grad_phi_z, grad_phi_abs_sq 
    gc.collect() 
    torch.cuda.empty_cache() 

    return total_energy_val


def compute_power_spectrum_lss(phi_cpu_np_array: np.ndarray, k_val_range: list,\
                               dx_val_param: float, N_grid_param: int) -> tuple[np.ndarray, np.ndarray]:
    "\"\"\"Computes the 3D power spectrum P(k) from the final phi field.\"\"\""
    if np.all(phi_cpu_np_array == 0):\
        return np.array([]), np.array([])

    # Clip to prevent extreme values before FFT, which can cause NaNs in Fourier space
    phi_clipped = np.clip(phi_cpu_np_array, -1e6, 1e6)
    rho_field_np = phi_clipped**2
    fourier_transform = fftn(rho_field_np.astype(np.float32))
    
    power_spectrum_raw_data = np.abs(fourier_transform)**2 / (N_grid_param**6) 

    kx_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    ky_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    kz_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    
    kxx_mesh, kyy_mesh, kzz_mesh = np.meshgrid(kx_coords, ky_coords, kz_coords, indexing='ij', sparse=True)
    k_magnitude_values = np.sqrt(kxx_mesh**2 + kyy_mesh**2 + kzz_mesh**2)

    k_bins_def = np.linspace(k_val_range[0], k_val_range[1], 50) 
    
    power_binned_values, _ = np.histogram(\
        k_magnitude_values.ravel(), bins=k_bins_def,\
        weights=power_spectrum_raw_data.ravel(), density=False\
    )
    counts_in_bins, _ = np.histogram(k_magnitude_values.ravel(), bins=k_bins_def)
    
    power_binned_final = np.divide(power_binned_values, counts_in_bins, out=np.zeros_like(power_binned_values), where=counts_in_bins!=0)
    k_bin_centers_final = (k_bins_def[:-1] + k_bins_def[1:]) / 2

    return k_bin_centers_final, power_binned_final


def compute_correlation_function_lss(phi_cpu_np_array: np.ndarray, dx_val_param: float,\
                                     N_grid_param: int, L_box_param: float) -> tuple[np.ndarray, np.ndarray]:
    "\"\"\"Computes the 3D correlation function xi(r) from the density field (phi^2).\"\"\""
    # Clip to prevent extreme values before FFT
    phi_clipped = np.clip(phi_cpu_np_array, -1e6, 1e6).astype(np.float32)
    rho_field_np = phi_clipped**2
    
    rho_mean = np.mean(rho_field_np)
    rho_k = fftn(rho_field_np - rho_mean)
    power_spectrum_rho = np.abs(rho_k)**2

    correlation_func_raw_data = ifftn(power_spectrum_rho).real 

    if rho_mean**2 > 1e-15:
        xi_normalized = correlation_func_raw_data / (N_grid_param**3 * rho_mean**2)
    else:
        xi_normalized = np.zeros_like(correlation_func_raw_data)

    indices_shifted = np.fft.ifftshift(np.arange(N_grid_param)) - (N_grid_param // 2)
    rx_coords = indices_shifted * dx_val_param
    ry_coords = indices_shifted * dx_val_param
    rz_coords = indices_shifted * dx_val_param
    rxx_mesh, ryy_mesh, rzz_mesh = np.meshgrid(rx_coords, ry_coords, rz_coords, indexing='ij', sparse=True)
    r_magnitude_values = np.sqrt(rxx_mesh**2 + ryy_mesh**2 + rzz_mesh**2)

    r_bins_def = np.linspace(0, L_box_param / 2, 50) 
    
    corr_binned_values, _ = np.histogram(\
        r_magnitude_values.ravel(), bins=r_bins_def,\
        weights=xi_normalized.ravel()\
    )
    counts_in_bins, _ = np.histogram(r_magnitude_values.ravel(), bins=r_bins_def)
    
    corr_binned_final = np.divide(corr_binned_values, counts_in_bins, out=np.zeros_like(corr_binned_values), where=counts_in_bins!=0)
    r_bin_centers_final = (r_bins_def[:-1] + r_bins_def[1:]) / 2

    return r_bin_centers_final, corr_binned_final

PyTorch version: 2.4.1+cu121
Number of GPUs available: 4, Available Devices: [device(type='cuda', index=0), device(type='cuda', index=1), device(type='cuda', index=2), device(type='cuda', index=3)]
Using compute device: cuda:0
System RAM: 988.75 GB
LSS Definitive Checkpoints will be saved to: ./EFM_Simulations/checkpoints/LSS_DEFINITIVE_N650_Run/
LSS Definitive Data/Plots will be saved to: ./EFM_Simulations/data/LSS_DEFINITIVE_N650_Run/


## Simulation Orchestration for Definitive Run

This section sets up the definitive high-resolution LSS simulation. It includes robust checkpointing and resume logic to ensure progress is saved and can be continued, which is crucial for managing computational resources.
The `run_lss_simulation` function directly calls `update_phi_rk4_lss`.

In [0]:
def run_lss_simulation(config: dict, device: torch.device, checkpoint_dir: str, data_dir: str):
    """Main simulation loop for EFM LSS with domain decomposition and multi-GPU support."""
    print(f"Initializing fields for EFM LSS simulation ({config['run_id']}) on {device} with {torch.cuda.device_count()} GPUs...")

    torch.manual_seed(42)
    np.random.seed(42)

    num_gpus = torch.cuda.device_count()
    devices = [torch.device(f'cuda:{i}') for i in range(num_gpus)] if num_gpus > 0 else [torch.device('cpu')]
    phi_subdomains = [None] * num_gpus
    phi_dot_subdomains = [None] * num_gpus
    start_step = 0
    energy_history = []
    # density_norm_history will be computed at the end in plot_lss_results from phi_final_cpu for simplicity.

    N = config['N']
    base_size = N // num_gpus
    remainder = N % num_gpus
    subdomain_sizes = [base_size + 1 if i < remainder else base_size for i in range(num_gpus)]
    subdomain_starts = [sum(subdomain_sizes[:i]) for i in range(num_gpus)]
    subdomain_ends = [start + size for start, size in zip(subdomain_starts, subdomain_sizes)]
    print(f"Grid size N: {N}, Subdomain sizes: {subdomain_sizes}")

    # Check for existing checkpoint to resume
    checkpoint_file_pattern = os.path.join(checkpoint_dir, f"CKPT_{config['run_id']}_step_*.npz")
    existing_checkpoints = sorted(glob.glob(checkpoint_file_pattern),\
                                 key=lambda f: int(os.path.basename(f).split('_step_')[1].split('.npz')[0]), reverse=True)
    
    if existing_checkpoints: 
        latest_checkpoint_file = existing_checkpoints[0]
        print(f"Resuming from checkpoint: {latest_checkpoint_file}")
        try: 
            checkpoint = np.load(latest_checkpoint_file, allow_pickle=True)
            phi_full = torch.from_numpy(checkpoint['phi_r_cpu']).to(dtype=torch.float16)
            phi_dot_full = torch.from_numpy(checkpoint['phi_dot_r_cpu']).to(dtype=torch.float16)
            start_step = checkpoint['last_step'].item() + 1
            if 'energy_history_saved' in checkpoint.files:
                energy_history.extend(checkpoint['energy_history_saved'].tolist())
            
            # Distribute loaded full field to subdomains
            for i in range(num_gpus):
                phi_subdomains[i] = torch.empty((subdomain_sizes[i], N, N), dtype=torch.float16, device=devices[i])
                phi_dot_subdomains[i] = torch.empty((subdomain_sizes[i], N, N), dtype=torch.float16, device=devices[i])
                # Use copy_() for non-blocking copy from CPU to GPU
                phi_subdomains[i].copy_(phi_full[subdomain_starts[i]:subdomain_ends[i]].to(devices[i], non_blocking=True))
                phi_dot_subdomains[i].copy_(phi_dot_full[subdomain_starts[i]:subdomain_ends[i]].to(devices[i], non_blocking=True))
            
            print(f"Resumed from step {start_step}. Last recorded energy: {energy_history[-1]:.4g}" if energy_history else f"Resumed from step {start_step}.")
            del checkpoint, phi_full, phi_dot_full
            gc.collect()
            torch.cuda.empty_cache()
        except Exception as e:
            print(f"Error loading checkpoint {latest_checkpoint_file}: {e}. Starting from scratch.")
            phi_subdomains = [None] * num_gpus # Reset to trigger new initialization if loading fails

    if all(p is None for p in phi_subdomains): 
        print("No valid checkpoint found or error loading. Starting simulation from scratch.")
        x_coords = np.linspace(-config['L_sim_unit']/2, config['L_sim_unit']/2, config['N'], dtype=np.float32)
        X, Y, Z = np.meshgrid(x_coords, x_coords, x_coords, indexing='ij')
        
        seeded_modes_field = config['seeded_perturbation_amplitude'] * (\
            np.sin(config['k_seed_primary'] * X) +\
            np.sin(config['k_seed_secondary'] * Y) +\
            np.cos(config['k_seed_primary'] * Z)\
        )
        random_background_noise = config['background_noise_amplitude'] * (np.random.rand(config['N'], config['N'], config['N']) - 0.5)
        initial_phi_np = seeded_modes_field + random_background_noise
        if np.all(initial_phi_np == 0): 
            initial_phi_np = config['background_noise_amplitude'] * (np.random.rand(config['N'], config['N'], config['N']) - 0.5)
        
        initial_phi_tensor = torch.from_numpy(initial_phi_np.astype(np.float16)).pin_memory()
        
        for i in range(num_gpus):
            phi_subdomains[i] = initial_phi_tensor[subdomain_starts[i]:subdomain_ends[i]].to(devices[i], non_blocking=True)
            phi_dot_subdomains[i] = torch.zeros((subdomain_sizes[i], N, N), dtype=torch.float16, device=devices[i])
        
        if start_step == 0:
            with torch.no_grad():
                phi_full_cpu = torch.cat([p.cpu() for p in phi_subdomains], dim=0) 
                phi_dot_full_cpu = torch.cat([p.cpu() for p in phi_dot_subdomains], dim=0)
                current_energy = compute_total_energy_lss(phi=phi_full_cpu, phi_dot=phi_dot_full_cpu, \
                                                       m_sq_param=config['m_sim_unit_inv']**2, \
                                                       g_param=config['g_sim'], eta_param=config['eta_sim'], \
                                                       dx=config['dx_sim_unit'], c_sq_param=config['c_sim_unit']**2)
                current_density_norm = torch.sum(phi_full_cpu.to(torch.float32)**2).item() * config['k_efm_gravity_coupling']
                del phi_full_cpu, phi_dot_full_cpu 
            energy_history.append(current_energy) 
            print(f"Initial State: Energy={current_energy:.4g}, Density Norm={current_density_norm:.4g}")

    efm_models = []
    streams = [torch.cuda.Stream(device=devices[i]) for i in range(num_gpus)] 
    for i in range(num_gpus):
        model = EFMLSSModule(\
            dx=config['dx_sim_unit'], m_sq=config['m_sim_unit_inv']**2, g=config['g_sim'], eta=config['eta_sim'],\
            k_gravity=config['k_efm_gravity_coupling'], G_gravity=config['G_sim_unit'], c_sq=config['c_sim_unit']**2,\
            alpha_param=config['alpha_sim'], delta_param=config['delta_sim']\
        ).to(devices[i])
        model.eval()
        with torch.no_grad():
            dummy_phi = torch.zeros((subdomain_sizes[i], N, N), dtype=torch.float16, device=devices[i])
            dummy_halo = torch.zeros((1, N, N), dtype=torch.float16, device=devices[i]) 
            model.conv_laplacian(dummy_phi, dummy_halo, dummy_halo)
        efm_models.append(model)
    
    sim_start_time = time.time()
    numerical_error = False

    halo_buffer_cpu_left = [torch.empty((1, N, N), dtype=torch.float16, pin_memory=True) for _ in range(num_gpus)]
    halo_buffer_cpu_right = [torch.empty((1, N, N), dtype=torch.float16, pin_memory=True) for _ in range(num_gpus)]
    
    def exchange_halos(phi_subs, phi_dot_subs): 
        # Send data to CPU buffers (for neighbor GPUs)
        for i in range(num_gpus):
            target_gpu_idx = (i + 1) % num_gpus
            source_gpu_idx = (i - 1 + num_gpus) % num_gpus 
            with torch.cuda.stream(streams[i]): 
                halo_buffer_cpu_right[target_gpu_idx].copy_(phi_subs[i][-1:, :, :].cpu(), non_blocking=True)
                halo_buffer_cpu_left[source_gpu_idx].copy_(phi_subs[i][0:1, :, :].cpu(), non_blocking=True)
        
        torch.cuda.synchronize() 

        halos_left = [None] * num_gpus
        halos_right = [None] * num_gpus
        for i in range(num_gpus):
            with torch.cuda.stream(streams[i]): 
                halos_right[i] = halo_buffer_cpu_right[i].to(devices[i], non_blocking=True)
                halos_left[i] = halo_buffer_cpu_left[i].to(devices[i], non_blocking=True)
        
        torch.cuda.synchronize() 
        
        return halos_left, halos_right

    # Main simulation loop
    for t_step in tqdm(range(start_step, config['T_steps']), desc=f"LSS Sim ({config['run_id']})"):
        # Check for NaN/Inf/extreme values in subdomains (more frequent for diagnostics)
        if (t_step + 1) % 1000 == 0 or t_step == start_step: 
            for i in range(num_gpus): 
                phi_max = torch.max(torch.abs(phi_subdomains[i])).item()
                phi_dot_max = torch.max(torch.abs(phi_dot_subdomains[i])).item()
                if phi_max > 1e4 or phi_dot_max > 1e4 or \
                   torch.any(torch.isinf(phi_subdomains[i])) or torch.any(torch.isnan(phi_subdomains[i])) or \
                   torch.any(torch.isinf(phi_dot_subdomains[i])) or torch.any(torch.isnan(phi_dot_subdomains[i])):
                    print(f"\nError: NaN/Inf or extreme values in subdomain {i} at step {t_step + 1}: phi_max={phi_max:.4g}, phi_dot_max={phi_dot_max:.4g}\n")
                    numerical_error = True # This line's indentation should be at the same level as the 'if' it belongs to
                    break 
            if numerical_error: 
                break 
        
        # Clip fields to prevent runaway growth (tighter clipping to maintain stability)
        with torch.no_grad(): 
            for i in range(num_gpus): 
                phi_subdomains[i].clamp_(-1e3, 1e3)  
                phi_dot_subdomains[i].clamp_(-1e3, 1e3)

        # Halo exchange for phi (simplified to phi only for conv_laplacian, as before)
        halos_left_phi, halos_right_phi = exchange_halos(phi_subdomains, phi_dot_subdomains) 

        with torch.no_grad(): 
            for i in range(num_gpus): 
                with torch.cuda.stream(streams[i]): 
                    phi_subdomains[i], phi_dot_subdomains[i] = update_phi_rk4_lss(\
                        phi_subdomains[i], phi_dot_subdomains[i], config['dt_sim_unit'], efm_models[i],\
                        halos_left_phi[i].squeeze(0), halos_right_phi[i].squeeze(0) 
                    )
        torch.cuda.synchronize() 

        # Record energy diagnostics periodically
        if (t_step + 1) % config['history_every_n_steps'] == 0: 
            with torch.no_grad():
                phi_full_cpu_diag = torch.cat([p.cpu() for p in phi_subdomains], dim=0)
                phi_dot_full_cpu_diag = torch.cat([p.cpu() for p in phi_dot_subdomains], dim=0)
                current_energy = compute_total_energy_lss(phi=phi_full_cpu_diag, phi_dot=phi_dot_full_cpu_diag,\
                                                       m_sq_param=efm_models[0].m_sq, g_param=efm_models[0].g, eta_param=efm_models[0].eta,\
                                                       dx=efm_models[0].dx, c_sq_param=efm_models[0].c_sq)
                current_density_norm = torch.sum(phi_full_cpu_diag.to(torch.float32)**2).item() * config['k_efm_gravity_coupling']
                del phi_full_cpu_diag, phi_dot_full_cpu_diag 
            energy_history.append(current_energy) 
            tqdm.write(f"Step {t_step+1}: E={current_energy:.3e}, DN={current_density_norm:.3e}")
            if np.isnan(current_energy) or np.isinf(current_energy): 
                print(f"Instability: Energy is NaN/Inf at step {t_step+1}. Stopping.")
                numerical_error = True 
                break 

        # Save intermediate checkpoint
        if (t_step + 1) % config['checkpoint_every_n_steps'] == 0 and (t_step + 1) < config['T_steps']: 
            intermediate_ckpt_file = os.path.join(checkpoint_dir, f"CKPT_{config['run_id']}_step_{t_step+1}.npz")
            try: 
                with torch.no_grad():
                    phi_full = torch.cat([p.cpu() for p in phi_subdomains], dim=0).numpy()
                    phi_dot_full = torch.cat([p.cpu() for p in phi_dot_subdomains], dim=0).numpy()
                np.savez_compressed(intermediate_ckpt_file,\
                                    phi_r_cpu=phi_full,\
                                    phi_dot_r_cpu=phi_dot_full,\
                                    last_step=t_step,\
                                    config_lss_saved=config,\
                                    energy_history_saved=np.array(energy_history))\
                print(f"Checkpoint saved at step {t_step+1} to {intermediate_ckpt_file}")
                del phi_full, phi_dot_full
                gc.collect()
                torch.cuda.empty_cache()
            except Exception as e_save:
                print(f"Error saving intermediate checkpoint: {e_save}")

    sim_duration = time.time() - sim_start_time 
    print(f"Simulation finished in {sim_duration:.2f} seconds.")
    if numerical_error: print("Simulation stopped due to numerical error.") 

    # Save final state and history
    final_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S") 
    final_data_filename = os.path.join(data_dir, f"FINAL_LSS_DATA_{config['run_id']}_{final_timestamp}.npz")
    with torch.no_grad():
        phi_final = torch.cat([p.cpu() for p in phi_subdomains], dim=0).numpy()
        phi_dot_final = torch.cat([p.cpu() for p in phi_dot_subdomains], dim=0).numpy()
    np.savez_compressed(final_data_filename,\
                        phi_final_cpu=phi_final,\
                        phi_dot_final_cpu=phi_dot_final,\
                        energy_history=np.array(energy_history),\
                        config_lss=config,\
                        sim_had_numerical_error=numerical_error)\
    print(f"Final LSS simulation data saved to {final_data_filename}")

    # Clean up all tensors and models
    del phi_subdomains, phi_dot_subdomains, efm_models, halo_buffer_cpu_left, halo_buffer_cpu_right, streams
    gc.collect()
    torch.cuda.empty_cache()

    return final_data_filename
